# Prep

## Download data

In [1]:
# Check working directory; this is the location where the 
# yolov5 model and TACO dataset will download.  

!pwd

/Users/ginadurante/Desktop/Data_sci/trashai/trash-ai-local-MBM1


In [ ]:
# Change to different directory if necessary by editing path below, then uncomment line and run cell

# %cd /path/to/project/dir  

In [ ]:
!test -d TACO || git clone https://github.com/pedropro/TACO
!test -d TACO

In [ ]:
# Change working directory to TACO subdirectory
%cd TACO

In [ ]:
# Run download script
%run download.py

In [ ]:
# Move back up to parent directory 
%cd ..

## Download yolov5 and required packages

In [ ]:
!test -d yolov5 || git clone https://github.com/ultralytics/yolov5
!test -f yolov5 && test -d yolov5
%cd yolov5
!pip install --upgrade pip
!pip install -r requirements.txt
!pip install split-folders wandb torch tensorboard opencv-python torchvision pycocotools 

## Import pycocotools, get categories, and set up dictionary

In [ ]:
# import pycocotools
from pycocotools.coco import COCO
import numpy as np
import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
annotation_path = '../TACO/data/annotations.json'

data_source = COCO(annotation_file=annotation_path)
catIds = data_source.getCatIds()
categories = data_source.loadCats(catIds)
categories.sort(key=lambda x: x['id'])

In [ ]:
categories  # Uncomment to view categories downloaded from TACO repo

In [ ]:
len(categories)

In [ ]:
classes = {}
taco_labels = {}
taco_labels_inverse = {}

for c in categories:
    taco_labels[len(classes)] = c['id']
    taco_labels_inverse[c['id']] = len(classes)
    classes[c['name']] = len(classes)


In [ ]:
len(classes)

## Split data for training, validation, testing

In [ ]:
# Check to make sure you're in yolov5 directory
!pwd

In [ ]:
%cd yolov5

In [ ]:
!mkdir -p tmp/labels tmp/images

In [ ]:
IMAGES_PATH = 'tmp/images/'
LABELS_PATH = 'tmp/labels/'
import shutil
import os
img_ids = data_source.getImgIds()

for index, img_id in tqdm.tqdm(enumerate(img_ids)):
    img_info = data_source.loadImgs(img_id)[0]
    # img_dir: batch_x/.....jpg ---> batch_x_......jpg
    img_dir = img_info['file_name'].replace('/', '_')
    
    image_name = img_dir.split('.')[0]
    label_dir = LABELS_PATH + image_name + '.txt'
    
    height = img_info['height']
    width = img_info['width']

    # get images
    shutil.copy(f"../TACO/data/{img_info['file_name']}", os.path.join(IMAGES_PATH, img_dir))

    # get labels
    with open(label_dir, mode='w') as fp:
        annotation_id = data_source.getAnnIds(img_id)
        if len(annotation_id) == 0:
            fp.write('')
            continue
        boxes = np.zeros((0, 5))
        annotations = data_source.loadAnns(annotation_id)
        lines = ''
        for annotation in annotations:
            label = taco_labels_inverse[annotation['category_id']]
            box = annotation['bbox']
            # some annotations have basically no width / height (extremely small), skip them
            if box[2] < 1 or box[3] < 1:
                continue
            # top_x,top_y,width,height ----> cen_x,cen_y,width,height
            # standardize to 0-1
            box[0] = round((box[0] + box[2] / 2) / width, 6)
            box[1] = round((box[1] + box[3] / 2) / height, 6)
            box[2] = round(box[2] / width, 6)
            box[3] = round(box[3] / height, 6)
            # line: label x_center y_center width height
            lines = lines + str(label)
            for i in box:
                lines += ' ' + str(i)
            lines += '\n'
        fp.writelines(lines)

In [ ]:
print(len(os.listdir(IMAGES_PATH)))
print(len(os.listdir(LABELS_PATH)))

In [ ]:
import splitfolders
splitfolders.ratio('tmp', output='taco', seed=1337, ratio=(.8, 0.1,0.1)) 

Check files:

In [ ]:
!pwd

In [ ]:
# Change to yolov5 directory if not still there
# %cd yolov5

In [ ]:
print(sorted(os.listdir('taco/train/images'))[:5])
print(sorted(os.listdir('taco/val/images'))[:5])
print(sorted(os.listdir('taco/test/images'))[:5])

Remove tmp fodler

In [ ]:
!rm -r tmp

Run this if the error "a view of a leaf Variable that requires grad is being used in an in-place operation", then train again

In [ ]:
# Run this only if the error "a view of a leaf Variable that requires grad is being used in an in-place operation", then train again

import time

import time
from pathlib import Path
import shutil

RUNS_DIR=Path("runs/train")
PRUN_DIR=Path(f'prev_runs/{time.strftime("%Y%m%d_%H%M%S")}')
PRUN_DIR.mkdir(parents=True, exist_ok=True)
for i in RUNS_DIR.glob('*'):
    print(f'moving previous run {i} to {PRUN_DIR}')
    shutil.move(str(i), str(PRUN_DIR))

# New Training ~ 6 hours on 16 i9 cores

In [ ]:
# Be forewarned, this takes ~6 hours training for 50 epochs on 16 i9 cores
import time
import shutil
start = time.time()
!wandb disabled
!python train.py --multi-scale \
    --img 640 \
    --batch 32 \
    --epochs 10 \
    --data ../taco.yaml   # change path to taco.yaml as needed

end = time.time()

In [ ]:
print('Training time:', end - start, 'seconds')

## Show the training result

In [ ]:
# Check working directory
!pwd

In [ ]:
# Switch to yolov5 directory if you're not already in it
# %cd /path/to/yolov5   # Change path name, uncomment this line, and run cell to switch to yolov5 directory

Now that you're in the yolov5 directory, change to the the runs/train/exp directory.  Note that if you have more than one model run, you may see multiple './exp' files, e.g., 'runs/train/exp', 'runs/train/exp2', 'runs/train/exp3', etc. in your local directory. Change the path name to the appropriate './exp' file to pull results from most recent successful run.

In [ ]:
# Change the path name to relevant './exp' file 
    
%cd 'runs/train/exp' 
%ls  

In [ ]:
res_path = 'results.png'
img = plt.imread(res_path)
plt.figure(figsize=(20, 20))
plt.imshow(img)
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
# %cd /path/to/yolov5   # change directory to make sure that you're in the yolov5 directory
!python detect.py \
   --weights runs/train/exp/weights/best.pt \
   --img 640 --conf 0.25 --source taco/test/images 

In [ ]:
OUT_PATH = 'runs/detect/exp'
cnt = 0
for file in os.listdir(OUT_PATH):
    img = plt.imread(os.path.join(OUT_PATH, file))
    plt.figure(figsize=(15, 15))
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    plt.show()
    cnt += 1

In [ ]:
# Generate web model  
# Note:  You will need to have Tensorflowjs installed on your machine to run this successfully.

import time
from pathlib import Path
import shutil
import os

BN="trained"
SRC_DIR=Path('runs/train/exp')
SAVE_PT=SRC_DIR.joinpath('weights/best.pt')
PRUN_DIR=Path('prev_runs')
TS=time.time()
DEST_DIR=Path(f'{PRUN_DIR}/{TS}')
DEST_DIR.mkdir(parents=True, exist_ok=True)
DEST_FILE=DEST_DIR.joinpath(f'{BN}.pt')
MODEL_DIR=DEST_DIR.joinpath(f'{TS}/{BN}_web_model')
print(f'moving previous run {SAVE_PT} to {DEST_FILE}')
shutil.move(str(SAVE_PT), str(DEST_FILE))
cmd = f'''
python export.py \
   --weights  {DEST_FILE} \
   --include tfjs 
'''
os.system(cmd)
shutil.rmtree(str(SRC_DIR))
